In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
import stumpy
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import numpy.testing as npt
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import os
from vnstock import * #import all functions, including functions that provide OHLC data for charting
# from vnstock.chart import * # import chart functions

In [3]:
# df = stock_historical_data("HAH", "2020-01-01", "2023-10-10", "1D", "stock")
# fig = candlestick_chart(df, ma_periods=[50,200], show_volume=False, reference_period=300, figure_size=(15, 8), 
#                         title='VIC - Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', 
#                         colors=('lightgray', 'gray'), reference_colors=('black', 'blue'))
# fig.show()

Sử dụng các tiêu chí lọc như sau để thiết lập tham số params.

- CANSLIM: epsGrowth1Year, lastQuarterProfitGrowth, roe, avgTradingValue20Day, relativeStrength1Month
- Giá trị: roe, pe, avgTradingValue20Day
- Cổ tức cao: dividendYield, avgTradingValue20Day
- Phá nền mua: avgTradingValue20Day, forecastVolumeRatio, breakout: 'BULLISH'
- Giá tăng + Đột biến khối lượng: avgTradingValue20Day, forecastVolumeRatio
- Vượt đỉnh 52 tuần: avgTradingValue20Day, priceBreakOut52Week: 'BREAK_OUT'
- Phá đáy 52 tuần: avgTradingValue20Day, priceWashOut52Week: 'WASH_OUT'
- Uptrend ngắn hạn: avgTradingValue20Day, uptrend: 'buy-signal'
- Vượt trội ngắn hạn: relativeStrength3Day,
- Tăng trưởng: epsGrowth1Year, roe, avgTradingValue20Day

In [4]:
def filter_stocks(param):
    df = stock_screening_insights (param, size=1700, drop_lang='vi')
    if len(df)!= 0:
        print(df.ticker.unique())
    return df

## canslim

In [5]:
canslim_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    "epsGrowth1Year": (25, 100),  # Minimum 1-year EPS growth
    "lastQuarterProfitGrowth": (25, 100),  # Minimum last quarter profit growth
    "roe": (20, 100),  # Minimum Return on Equity (ROE)
    "avgTradingValue20Day": (50, 1000),  # Minimum 20-day average trading value
}
df = filter_stocks(canslim_params)
df.head()

""


In [6]:
params = {
            "exchangeName": "HOSE",
            "marketCap": (1_000, 200_000),
            'uptrend': 'buy-signal',
            'tcbsBuySellSignal':'BUY',
            "avgTradingValue20Day": (10, 100_000),
            'priceNearRealtime': (15,80),
        }
df = stock_screening_insights (params, size=1700, drop_lang='vi')
df.head()

,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,uptrend.en,priceVsSMA20.en,foreignTransaction.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,bollingBandSignal.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,tcbsRecommend.en,priceVsSma10.en,sarVsMacdHist.en
0,CTG,,137204,16.0,4.0,3.1,3.8,3.4,0.3,1.3,64.1,36.3,18.2,1.1,7.8,1.2,NaN,0.0,20.62,14.47,19.07,17.68,NaN,22.40,NaN,76.63,84.22,70.96,71,74,39,56,84.44,29.25,53.0,27.16,60,-1409,1,3773,1.4,1.1,1.2,0.7,6.75,-0.85,2.45,-0.85,22.64,69.09,15,NaN,NaN,18133.2,0.4,11.5,15.7,12.0,-13.1,-28.9,31.5,383.3,HOSE,Banks,Buy Signal,Price cross above SMA(20),Buy > Sell,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Buy signal,Neutral,Yes,Price below SMA(100),NaN,NaN,NaN
1,DBC,,4816,-1.3,3.5,1.5,3.9,2.0,0.2,1.2,66.4,37.8,70.3,2.0,-84.9,1.1,12.6,0.0,6.89,14.57,-99.37,-58.46,10.36,0.46,1.1,80.60,77.69,74.75,95,35,20,76,138.90,21.25,51.9,5.79,56,309,3,-250,2.5,2.1,2.0,1.6,14.25,-5.13,6.78,-5.13,55.68,141.38,70,-97.9,-38.6,-60.6,-24.0,17.1,-94.0,2190.8,-25.0,-43.9,116.4,1343.7,HOSE,Food & Beverage,Buy Signal,Price cross above SMA(20),Buy < Sell,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Buy signal,Neutral,Yes,Price below SMA(100),NaN,NaN,NaN
2,FTS,,7853,14.4,3.9,4.4,3.0,3.6,0.0,2.1,63.7,27.7,5.3,1.4,17.0,2.3,15.7,0.0,-38.54,21.85,-62.38,4.08,71.02,56.49,0.7,77.98,86.39,101.46,94,15,93,96,110.00,39.15,51.0,28.10,74,1041,3,2301,1.9,1.4,1.2,1.2,15.15,-15.26,6.97,-15.26,132.68,329.29,55,-19.9,-21.4,493.8,497.4,21.7,NaN,87.4,-18.6,-31.5,255.1,656.9,HOSE,Financial Services,Buy Signal,Price cross above SMA(20),Buy < Sell,Buy,MACD Histogram < 0 and increase,Price below SMA(50),Buy signal,Neutral,Yes,Price below SMA(100),Sell,NaN,NaN
3,GEX,,16647,3.5,3.8,2.7,3.4,3.2,0.3,1.7,57.6,52.0,7.1,2.1,40.9,1.4,5.7,0.0,12.28,21.77,-64.48,-11.89,18.63,1.63,0.9,320.35,314.68,320.80,97,77,37,90,556.01,20.90,53.5,13.25,75,4686,1,511,2.5,2.0,1.9,1.3,15.79,-3.02,6.91,-3.02,78.63,29.34,70,-89.7,-27.5,434.9,6.7,-11.8,740.8,2359.2,-20.5,-56.6,94.4,222.3,HOSE,Industrial Goods & Services,Buy Signal,Price cross above SMA(20),Buy < Sell,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Buy signal,Neutral,Yes,Price below SMA(100),NaN,NaN,NaN
4,GVR,,73800,4.6,3.9,2.5,3.9,4.8,0.1,1.5,62.8,46.1,50.1,1.7,33.8,1.6,16.4,0.0,-2.92,4.87,-7.68,2.00,19.93,5.05,0.1,31.59,37.81,52.20,83,47,30,80,48.46,19.70,51.6,0.32,60,1781,1,583,2.1,1.4,1.0,0.8,13.87,-4.83,6.78,-4.83,50.96,114.91,4,-0.9,-0.9,2332.5,6.0,-25.2,-62.5,-45.1,-16.9,-53.2,111.8,199.7,HOSE,Chemicals,Buy Signal,Price cross above SMA(20),Buy > Sell,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Buy signal,Neutral,Yes,Price below SMA(100),NaN,Price cross above SMA(10),Buy signal


## Payback time

In [7]:
# https://www.youtube.com/watch?v=4YPyxfXML0A&t=1825s
paybacktime_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    'revenueGrowth5Year': (15,100),
    'pe':(10,20),
    "epsGrowth5Year": (10, 100),  # Minimum 1-year EPS growth
    "roe": (15, 100),  # Minimum Return on Equity (ROE)
    
    'priceNearRealtime': (15,80),
#     "lastQuarterProfitGrowth": (2, 100),  # Minimum last quarter profit growth
    "avgTradingValue20Day": (10, 2000),  # Minimum 20-day average trading value
#     "breakout": 'BULLISH',  # Only buy stocks when the market is in an uptrend
}
df = filter_stocks(paybacktime_params)
df.head()

['CTR' 'DCM' 'DGW' 'DHC' 'PNJ']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA20.en,foreignTransaction.en,tcbsRecommend.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,sarVsMacdHist.en,bollingBandSignal.en,uptrend.en,priceVsSma10.en
0,CTR,,8842,29.9,3.9,3.9,3.9,4.0,0.3,1.4,70.0,7.6,61.4,1.6,18.4,5.0,11.3,2.0,25.82,23.19,18.03,25.01,7.54,4.56,0.8,14.55,15.22,18.31,60,73,86,86,21.80,79.90,58.4,10.13,76,124,1,4346,2.1,1.6,1.3,0.7,8.56,-0.75,3.36,-0.75,78.45,844.99,35,-11.1,-14.2,497.2,18.3,21.5,9.6,20.6,-6.7,-15.9,130.8,909.2,HOSE,Construction & Materials,Price above SMA(20),Buy > Sell,Wait to buy,Strong buy,MACD Histogram > 0 and increase,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),NaN,NaN,NaN,NaN
1,DCM,,15670,16.8,4.7,4.3,4.0,5.0,-0.4,1.1,69.9,3.0,15.1,0.9,10.3,1.7,5.2,12.4,61.35,22.61,136.69,46.58,5.89,2.45,0.0,122.23,124.70,169.29,87,12,80,50,110.71,31.40,51.0,10.97,57,748,2,3050,1.1,0.9,0.7,0.7,12.14,-15.14,6.08,-15.14,18.32,325.28,25,12.8,13.7,1614.7,-9.0,-19.4,-89.9,-69.4,-17.1,-24.7,52.7,590.5,HOSE,Chemicals,Price cross above SMA(20),Buy > Sell,NaN,Strong buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),Buy signal,Buy signal,NaN,NaN
2,DGW,,7836,17.3,3.9,3.9,3.5,4.0,-0.9,1.9,68.3,52.2,20.3,0.9,19.9,3.2,14.5,2.7,5.28,41.96,4.49,35.51,7.11,1.89,0.8,68.39,81.94,94.75,87,10,41,24,69.62,50.10,47.7,23.66,40,2762,1,2517,1.2,0.9,0.8,0.7,14.38,-14.39,6.82,-14.39,-2.79,703.22,55,-6.0,-6.7,420.6,-10.8,-6.4,-43.2,-39.4,-20.3,-41.5,85.2,1716.8,HOSE,Retail,Price below SMA(20),Buy < Sell,NaN,Neutral,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,NaN,Buy Signal,Price cross above SMA(10)
3,DHC,,3308,17.3,4.3,3.8,2.8,4.4,-0.6,0.6,69.0,15.2,5.0,0.4,10.7,1.8,10.2,9.5,-5.51,37.18,-21.17,32.68,13.46,7.04,0.3,11.67,11.03,15.73,39,33,79,33,4.37,41.95,49.8,35.27,46,972,1,3929,0.4,0.4,0.3,0.3,9.24,-6.78,2.07,-6.78,7.76,123.03,70,-12.6,-14.6,316.3,-18.1,-19.4,-13.3,-19.0,-14.3,-54.8,78.1,3061.0,HOSE,Basic Resources,Price cross above SMA(20),Buy < Sell,NaN,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,Buy signal,NaN,NaN
4,PNJ,,24895,20.3,4.0,4.1,4.0,4.4,0.3,0.6,63.7,79.9,9.6,0.6,14.0,2.7,10.8,2.3,73.31,25.28,75.96,11.61,17.31,3.66,0.2,49.33,40.97,35.48,70,60,54,38,31.05,77.00,55.2,49.00,56,-10,1,5506,0.7,0.8,0.9,0.8,6.94,-3.09,1.45,-3.09,8.70,14.63,85,-4.9,-9.4,1805.9,-6.1,-17.4,0.4,-8.0,-17.5,-27.5,17.0,1706.9,HOSE,Personal & Household Goods,Price above SMA(20),NaN,NaN,Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,NaN,NaN,NaN


## Tang Truong

In [8]:
tangtruong_params = {
    "exchangeName": "HOSE",
    "marketCap": (1_000, 200_000),
    "epsGrowth1Year": (10, 100),  # Minimum 1-year EPS growth
    "roe": (15, 100),  # Minimum Return on Equity (ROE)
    'priceNearRealtime': (15,80),
    "avgTradingValue20Day": (10, 2000),  # Minimum 20-day average trading value
}
df = filter_stocks(tangtruong_params)
df.head()

['ACB' 'CTG' 'CTR' 'GAS' 'GMD' 'HAH' 'HDB' 'MBB' 'PHR' 'PNJ' 'REE' 'SAB'
 'STB' 'TPB' 'VGC' 'VHC' 'VIB' 'VPI']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,numDecreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,priceVsSMA20.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,uptrend.en,foreignTransaction.en,bollingBandSignal.en,tcbsRecommend.en,sarVsMacdHist.en,priceBreakOut52Week.en,dmiSignal.en,priceVsSma10.en,priceVsSMA5.en
0,ACB,,85838,24.3,4.2,4.6,3.9,3.6,0.2,1.1,69.3,25.1,33.7,1.9,5.9,1.3,NaN,5.5,22.18,20.27,42.54,35.07,NaN,47.93,NaN,107.00,113.51,107.77,63,84,59,62,177.60,22.80,61.0,30.00,67,0,1,NaN,3839,2.4,1.8,1.8,1.5,5.56,0.44,3.17,0.44,28.15,104.80,90,NaN,NaN,14909.0,18.8,15.9,12.6,-2.0,-6.6,-24.9,52.5,697.7,HOSE,Banks,Price above SMA(20),Strong buy,MACD Histogram > 0 and increase,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CTG,,137204,16.0,4.0,3.1,3.8,3.4,0.3,1.3,64.1,36.3,18.2,1.1,7.8,1.2,NaN,0.0,20.62,14.47,19.07,17.68,NaN,22.40,NaN,76.63,84.22,70.96,71,74,39,56,84.44,29.25,53.0,27.16,60,-1409,1,NaN,3773,1.4,1.1,1.2,0.7,6.75,-0.85,2.45,-0.85,22.64,69.09,15,NaN,NaN,18133.2,0.4,11.5,15.7,12.0,-13.1,-28.9,31.5,383.3,HOSE,Banks,Price cross above SMA(20),Buy,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),Buy Signal,Buy > Sell,Buy signal,NaN,NaN,NaN,NaN,NaN,NaN
2,CTR,,8842,29.9,3.9,3.9,3.9,4.0,0.3,1.4,70.0,7.6,61.4,1.6,18.4,5.0,11.3,2.0,25.82,23.19,18.03,25.01,7.54,4.56,0.8,14.55,15.22,18.31,60,73,86,86,21.80,79.90,58.4,10.13,76,124,1,NaN,4346,2.1,1.6,1.3,0.7,8.56,-0.75,3.36,-0.75,78.45,844.99,35,-11.1,-14.2,497.2,18.3,21.5,9.6,20.6,-6.7,-15.9,130.8,909.2,HOSE,Construction & Materials,Price above SMA(20),Strong buy,MACD Histogram > 0 and increase,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),NaN,Buy > Sell,NaN,Wait to buy,NaN,NaN,NaN,NaN,NaN
3,GAS,,180983,20.5,4.1,3.1,4.5,5.0,-0.5,0.9,66.2,7.8,62.1,1.3,15.1,3.0,10.5,4.4,27.51,9.32,70.63,4.95,16.64,10.74,0.1,33.60,34.91,44.42,41,34,65,14,39.91,80.00,45.5,2.89,38,-1089,1,NaN,5283,1.5,1.3,1.0,0.9,4.03,-6.98,1.52,-6.98,-13.05,-7.30,5,2.6,5.5,12133.1,-9.1,-13.1,-21.5,-38.0,-17.1,-34.3,8.6,281.1,HOSE,Utilities,Price below SMA(20),Neutral,MACD Histogram > 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,Buy > Sell,NaN,Wait to buy,NaN,NaN,NaN,NaN,NaN
4,GMD,,20220,29.5,3.8,3.0,4.3,4.6,-0.1,0.7,76.5,19.5,13.3,1.2,9.0,2.5,13.7,4.5,21.58,-0.43,62.36,14.03,46.57,25.47,0.2,109.07,89.06,90.11,70,82,95,72,115.81,67.80,62.3,48.77,80,2666,6,NaN,7504,1.3,1.4,1.3,1.0,9.71,2.26,2.57,2.26,50.04,172.98,90,-4.3,-6.6,2295.5,0.6,-6.7,4.2,471.7,-0.3,2.3,92.1,1606.3,HOSE,Industrial Goods & Services,Price above SMA(20),Strong buy,MACD Histogram > 0 and increase,Price above SMA(50),Neutral,Yes,Price above SMA(100),NaN,Buy > Sell,NaN,Wait to buy,NaN,NaN,NaN,NaN,NaN


## my params

In [9]:
my_params = {
    "exchangeName": "HOSE,HNX",
    "marketCap": (1_000, 200_000),
    "roe": (10, 100),  # Minimum Return on Equity (ROE)
    'pe': (10,20),
    'priceNearRealtime': (10,100),
    "avgTradingValue20Day": (100, 1000),  # Minimum 20-day average trading value
#     'uptrend': 'buy-signal',
    'macdHistogram': 'macdHistLT0Increase',
    'strongBuyPercentage': (20,100),
    'relativeStrength3Day': (50,100),
}
df = filter_stocks(my_params)
df.head()

['FTS' 'IDC']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,uptrend.en,priceVsSMA20.en,foreignTransaction.en,tcbsRecommend.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,bollingBandSignal.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,priceVsSma10.en
0,FTS,,7853,14.4,3.9,4.4,3,3.6,0.0,2.1,63.7,27.7,5.3,1.4,17.0,2.3,15.7,0,-38.54,21.85,-62.38,4.08,71.02,56.49,0.7,77.98,86.39,101.46,94,15,93,96,110.00,39.15,51.0,28.10,74,1041,3,2301,1.9,1.4,1.2,1.2,15.15,-15.26,6.97,-15.26,132.68,329.29,55,-19.9,-21.4,493.8,497.4,21.7,NaN,87.4,-18.6,-31.5,255.1,656.9,HOSE,Financial Services,Buy Signal,Price cross above SMA(20),Buy < Sell,Sell,Buy,MACD Histogram < 0 and increase,Price below SMA(50),Buy signal,Neutral,Yes,Price below SMA(100),NaN
1,IDC,,14850,22.0,4.2,3.8,4,4.4,-0.7,2.0,65.0,69.1,0.2,1.4,15.2,3.6,6.3,0,74.03,8.75,289.05,36.84,28.07,11.15,0.6,128.92,171.33,209.37,88,76,80,66,157.35,48.20,54.2,15.11,78,7584,1,3177,1.6,1.0,0.8,0.9,12.09,-1.23,7.11,-1.23,34.89,207.93,65,-13.9,-12.2,1048.5,-29.7,-27.2,-61.9,-62.6,-9.4,-41.7,124.1,279.6,HNX,Real Estate,Buy Signal,Price cross above SMA(20),Buy > Sell,NaN,Buy,MACD Histogram < 0 and increase,Price cross above SMA(50),Buy signal,Neutral,Yes,Price above SMA(100),Price cross above SMA(10)


## Trend

In [10]:
trend_params = {
    "exchangeName": "HOSE,HNX",
    "marketCap": (1_000, 200_000),
    "epsGrowth5Year": (20, 100),  # Minimum 1-year EPS growth
#     "roe": (10, 100),  # Minimum Return on Equity (ROE)
#     'pe': (10,20),
#     'pb': (0,4),
    'priceNearRealtime': (10,100),
    'macdHistogram': 'macdHistLT0Increase',
    'strongBuyPercentage': (20,100),
    "avgTradingValue20Day": (100, 100_000),  # Minimum 20-day average trading value
    # 'uptrend': 'buy-signal',
}
df = filter_stocks(trend_params)
df.head()

['DPM' 'IDC']


,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,activeBuyPercentage,strongBuyPercentage,suddenlyHighVolumeMatching,forecastVolumeRatio,pe,pb,evEbitda,dividendYield,revenueGrowth1Year,revenueGrowth5Year,epsGrowth1Year,epsGrowth5Year,grossMargin,netMargin,doe,avgTradingValue5Day,avgTradingValue10Day,avgTradingValue20Day,relativeStrength3Day,relativeStrength1Month,relativeStrength3Month,relativeStrength1Year,totalTradingValue,priceNearRealtime,rsi14,foreignVolumePercent,tcRS,foreignBuySell20Session,numIncreaseContinuousDay,eps,volumeVsVSma5,volumeVsVSma10,volumeVsVSma20,volumeVsVSma50,priceGrowth1Week,priceGrowth1Month,prev1DayGrowthPercent,prev1MonthGrowthPercent,prev1YearGrowthPercent,prev5YearGrowthPercent,freeTransferRate,netCashPerMarketCap,netCashPerTotalAssets,profitForTheLast4Quarters,lastQuarterRevenueGrowth,secondQuarterRevenueGrowth,lastQuarterProfitGrowth,secondQuarterProfitGrowth,percent1YearFromPeak,percentAwayFromHistoricalPeak,percent1YearFromBottom,percentOffHistoricalBottom,exchangeName.en,industryName.en,uptrend.en,priceVsSMA20.en,foreignTransaction.en,tcbsBuySellSignal.en,macdHistogram.en,priceVsSma50.en,rsi14Status.en,hasFinancialReport.en,priceVsSMA100.en,priceVsSma10.en,bollingBandSignal.en
0,DPM,,12268,13,4.8,4.2,3.8,5.0,0.1,1.1,58.5,56.3,14.4,0.9,8.2,1.1,6.3,19.4,45.68,18.43,78.53,51.62,12.69,1.99,0.1,91.23,94.01,126.69,67,7,43,21,85.18,32.85,45.4,10.33,34,-5804,1,4017,1.1,1,0.8,0.7,13.86,-16.41,4.78,-16.41,-6.01,139.17,45,4.4,3.8,1572.2,-17.2,-26.1,-93.6,-92.4,-21.4,-43.4,21.8,331.4,HOSE,Chemicals,Buy Signal,Price below SMA(20),Buy > Sell,Neutral,MACD Histogram < 0 and increase,Price below SMA(50),Neutral,Yes,Price below SMA(100),NaN,NaN
1,IDC,,14850,22,4.2,3.8,4.0,4.4,-0.7,2.0,65.0,69.1,0.2,1.4,15.2,3.6,6.3,0.0,74.03,8.75,289.05,36.84,28.07,11.15,0.6,128.92,171.33,209.37,88,76,80,66,157.35,48.20,54.2,15.11,78,7584,1,3177,1.6,1,0.8,0.9,12.09,-1.23,7.11,-1.23,34.89,207.93,65,-13.9,-12.2,1048.5,-29.7,-27.2,-61.9,-62.6,-9.4,-41.7,124.1,279.6,HNX,Real Estate,Buy Signal,Price cross above SMA(20),Buy > Sell,Buy,MACD Histogram < 0 and increase,Price cross above SMA(50),Neutral,Yes,Price above SMA(100),Price cross above SMA(10),Buy signal
